### Plotting training stats

In this notebook we plot the evolutions of the filter parameters during model training. Note that reading the lightning logs requires tensorflow to be installed. Since there are conflicting requriements in pytorch and tensorflow I reccomend using a new conda environmen if you want tu run this notebook. 
```
conda create -n mario-tf python=3.9
conda acitvate mario-tf
pip install tensorflow ipython jupyter plotly pandas numpy
jupyter notebook
```

In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import src.constants as const
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import numpy as np
import pandas as pd
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
%load_ext autoreload
%autoreload 2

In [ ]:
def get_df_from_selection_bias_logs(version_str='lightning_logs/mario-const-speed/'):
    dirs = [d for d in os.listdir(version_str) if 'selection_bias' in d]
    event_file_paths = []
    for d in dirs:
        file = os.listdir(f'{version_str}{d}')[0]
        event_file_paths.append(f'{version_str}{d}/{file}')
    data = {}
    for p, d in zip(event_file_paths, dirs):
        val = []
        for event in tf.train.summary_iterator(p):
            for value in event.summary.value:
                if value.HasField('simple_value'):
                    val.append(value.simple_value)
        data.update({d:val})
    return pd.DataFrame(data)

In [ ]:
df_bias = get_df_from_selection_bias_logs()
df_bias.head()

In [ ]:
decoder_strs = [f'dec{c}' for c in range(len(const.HIDDEN_STATE_COLS))]
decoder_strs_dct = {d: sorted([c for c in df_bias.columns if d in c]) for d in decoder_strs}
decoder_strs_dct

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)
colors = ['#1f77b4','#ff7f0e', '#2ca02c', '#d62728']
for i, d in enumerate(decoder_strs):
    for c, color in zip(decoder_strs_dct[d], colors):
        fig.add_trace(go.Scatter(x=df_bias.index, y=df_bias[c],
                            mode='lines',
                            marker_color=color,
                            name=c.replace('_', ' ')), row=1, col=i+1)
fig.update_xaxes(title_text='Training step')
fig.update_layout(title_text="Selection bias during training")
fig.update_yaxes(title_text="Selection bias activation", col=1)
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_yaxes=True)
for i, d in enumerate(decoder_strs):
    for c, color in zip(decoder_strs_dct[d], colors):
        fig.add_trace(go.Scatter(x=df_bias.index, y=df_bias[c],
                            mode='lines',
                            marker_color=color,
                            name=c.replace('_', ' ')), row=1+i, col=1)
    fig.update_yaxes(title_text=f"Selection bias decoder {i}", col=1, row=i+1)
fig.update_xaxes(title_text='Training step')
fig.update_layout(title_text="Selection bias during training", height=1000)
fig.show()